In [5]:
import os
import json

INPUT_FILE = r'../carga_didatica/2025-1/formularios_combinados.json'
OUTPUT_PATH = r'../carga_didatica/2025-1/formularios'

In [6]:
def processar_formularios_compilados(turmasNoturnas: list):
    """
    Lê o arquivo JSON compilado e o divide em arquivos JSON individuais
    para cada docente, conforme a estrutura especificada.
    """
    
    # 1. Garantir que o diretório de saída exista
    try:
        os.makedirs(OUTPUT_PATH, exist_ok=True)
        print(f"Diretório de saída verificado/criado: '{OUTPUT_PATH}'")
    except OSError as e:
        print(f"Erro ao criar o diretório '{OUTPUT_PATH}': {e}")
        return

    # 2. Ler o arquivo de entrada compilado
    try:
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Erro: Arquivo de entrada não encontrado em '{INPUT_FILE}'")
        return
    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{INPUT_FILE}' não contém um JSON válido.")
        return
    except Exception as e:
        print(f"Erro inesperado ao ler o arquivo: {e}")
        return

    # 3. Criar um mapa de docentes para busca rápida (O(N))
    #    Isso é muito mais eficiente do que percorrer a lista 'docentes'
    #    a cada iteração dos 'formularios'.
    docentes_map = {}
    if 'docentes' in data and isinstance(data['docentes'], list):
        for docente_info in data['docentes']:
            if 'nome' in docente_info:
                docentes_map[docente_info['nome']] = docente_info
            else:
                print("Aviso: Encontrada entrada em 'docentes' sem a chave 'nome'.")

    # 4. Verificar se a chave 'formularios' existe
    if 'formularios' not in data or not isinstance(data['formularios'], dict):
        print("Erro: A chave 'formularios' não foi encontrada ou não é um dicionário no arquivo de entrada.")
        return

    # 5. Processar cada docente em 'formularios'
    total_processados = 0
    for nome_docente, formulario_docente in data['formularios'].items():
        
        # Buscar informações adicionais no mapa de docentes
        info_adicional = docentes_map.get(nome_docente)
        
        comentario = ""
        preferencia = ""
        
        if info_adicional:
            # Usamos .get() para evitar erros caso a chave não exista
            comentario = info_adicional.get('comentario', '')
            preferencia = info_adicional.get('agrupar', '')
        else:
            print(f"Aviso: Docente '{nome_docente}' encontrado em 'formularios' mas não na lista 'docentes'. 'comentario' e 'preferencia' ficarão vazios.")

        # 6. Montar a estrutura de saída padrão
        output_data = {
            nome_docente: {
                "formularios": formulario_docente,
                "avaliacao": {
                    "nota": -1,
                    "comentario": comentario,
                    "preferencia": preferencia
                },
                "semNoturna": not any(turma in turmasNoturnas for turma in formulario_docente.keys())
            }
        }
        
        # 7. Definir o nome do arquivo de saída e salvar
        output_filename = f"{nome_docente}.json"
        full_output_path = os.path.join(OUTPUT_PATH, output_filename)
        
        try:
            with open(full_output_path, 'w', encoding='utf-8') as f_out:
                # ensure_ascii=False -> Preserva caracteres como "José"
                # indent=2         -> Cria um arquivo formatado (bonito)
                json.dump(output_data, f_out, ensure_ascii=False, indent=2)
            total_processados += 1
        except IOError as e:
            print(f"Erro ao escrever o arquivo '{full_output_path}': {e}")
        except Exception as e:
             print(f"Erro inesperado ao processar '{nome_docente}': {e}")

    print(f"\nProcessamento concluído.")
    print(f"{total_processados} arquivos gerados em '{OUTPUT_PATH}'.")

In [7]:
turmas: list[dict] = json.load(open('public/turmas.json', 'r',  encoding='utf-8'))
turmasNoturnas = [turma['id'] for turma in turmas if turma['noturna']]

In [8]:
processar_formularios_compilados(turmasNoturnas)

Diretório de saída verificado/criado: '../carga_didatica/2025-1/formularios'

Processamento concluído.
37 arquivos gerados em '../carga_didatica/2025-1/formularios'.
